<a href="https://colab.research.google.com/github/yasminela/AI-ML/blob/main/Extract_feature_importances_from_the_XGBoost_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
import optuna

# Load the dataset
df = pd.read_csv('your_dataset.csv')  # Replace with your actual file path

# Separate features (X) and target (y)
X = df.drop(columns=['disorder'])  # Replace 'disorder' with your target column name
y = df['disorder']

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Identify categorical columns (all features are assumed to be categorical here)
categorical_features = X.columns.tolist()

ModuleNotFoundError: No module named 'category_encoders'

In [ ]:
# Define preprocessing steps for categorical features
categorical_transformer = Pipeline(steps=[
    ('target_encoder', TargetEncoder(smoothing=10)),  # Smoothed target encoding
])

# Combine transformers into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Apply preprocessing to the dataset
X_preprocessed = preprocessor.fit_transform(X, y_encoded)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

NameError: name 'Pipeline' is not defined

In [ ]:
def objective(trial):
    # Define hyperparameters for XGBoost
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'random_state': 42,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }

    # Train the model
    model = XGBClassifier(**params)
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Get the best parameters
best_params = study.best_params
print("Best Parameters:", best_params)

# Train the final model with the best parameters
best_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
best_model.fit(X_train, y_train)

NameError: name 'optuna' is not defined

In [ ]:
# Perform stratified k-fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(best_model, X_preprocessed, y_encoded, cv=cv, scoring='accuracy')

print("Cross-Validation Accuracy Scores:", cv_scores)
print("Mean CV Accuracy:", np.mean(cv_scores))

In [ ]:
# Extract feature importances from the XGBoost model
feature_importances = best_model.feature_importances_

# Match feature importances with feature names
feature_names = categorical_features

feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

print(feature_importance_df.head(10))  # Show top 10 features

In [ ]:
from sklearn.ensemble import VotingClassifier

# Define base models
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)
xgb_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')

# Create a voting classifier
voting_model = VotingClassifier(estimators=[('rf', rf_model), ('xgb', xgb_model)], voting='soft')

# Train the ensemble model
voting_model.fit(X_train, y_train)

# Evaluate the ensemble model
y_pred_ensemble = voting_model.predict(X_test)
print("\nEnsemble Classification Report:")
print(classification_report(y_test, y_pred_ensemble, target_names=label_encoder.classes_))